In [1]:
import sys, numpy as np
from keras.datasets import mnist

In [8]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()
images, labels = (x_train[0:1000].reshape(1000,28*28)/ 255, y_train[0:1000])
one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
  one_hot_labels[i][l] = 1
labels = one_hot_labels
test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
  test_labels[i][l] = 1
np.random.seed(1)
relu = lambda x:(x>=0) * x
relu2deriv = lambda x: x>=0
alpha, iterations, hidden_size, pixels_per_image, num_labels = (0.005, 350, 40, 784, 10)
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
  error, correct_cnt = (0.0, 0)

  for i in range(len(images)):
    layer_0 = images[i:i+1]
    layer_1 = relu(np.dot(layer_0,weights_0_1))
    layer_2 = np.dot(layer_1,weights_1_2)
    error += np.sum((labels[i:i+1] - layer_2) ** 2)
    correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))
    layer_2_delta = (labels[i:i+1] - layer_2)
    layer_1_delta = layer_2_delta.dot(weights_1_2.T)* relu2deriv(layer_1)
    weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
    weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
  sys.stdout.write("\r"+ \
                " I:"+str(j)+ \
                " Error:" + str(error/float(len(images)))[0:5] +\
                " Correct:" + str(correct_cnt/float(len(images))))

11501568/11490434 [==============================] - 0s 0us/step
 I:349 Error:0.108 Correct:1.0

In [12]:
if(j % 10 == 0 or j == iterations-1):
  error, correct_cnt = (0.0, 0)
  for i in range(len(test_images)):
    layer_0 = test_images[i:i+1]
    layer_1 = relu(np.dot(layer_0,weights_0_1))
    layer_2 = np.dot(layer_1,weights_1_2)
    error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
    correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
  sys.stdout.write(" Test-Err:" + str(error/float(len(test_images)))[0:5] +" Test-Acc:" + str(correct_cnt/float(len(test_images))))
  print()


 Test-Err:0.653 Test-Acc:0.7073


In [14]:
i = 0
layer_0 = images[i:i+1]
dropout_mask = np.random.randint(2,size=layer_1.shape)

layer_1 *= dropout_mask * 2
layer_2 = np.dot(layer_1, weights_1_2)

error += np.sum((labels[i:i+1] - layer_2) ** 2)
correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i+i+1]))

layer_2_delta = (labels[i:i+1] - layer_2)
layer_1_delta = layer_2_delta.dot(weights_1_2.T)* relu2deriv(layer_1)

layer_1_delta *= dropout_mask
weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

In [16]:
import numpy, sys
np.random.seed(1)

def relu(x):
  return (x >= 0) * x
def relu2deriv(output):
  return output >= 0
alpha, iterations, hidden_size = (0.005, 300, 100)
pixels_per_image, num_labels = (784, 10)
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1
for j in range(iterations):
  error, correct_cnt = (0.0,0)
  for i in range(len(images)):
    layer_0 = images[i:i+1]
    layer_1 = relu(np.dot(layer_0,weights_0_1))
    dropout_mask = np.random.randint(2, size=layer_1.shape)
    layer_1 *= dropout_mask * 2
    layer_2 = np.dot(layer_1,weights_1_2)
    error += np.sum((labels[i:i+1] - layer_2) ** 2)
    correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))
    layer_2_delta = (labels[i:i+1] - layer_2)
    layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
    layer_1_delta *= dropout_mask
    weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
    weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
  if(j%10 == 0):
    test_error = 0.0
    test_correct_cnt = 0

  for i in range(len(test_images)):
    layer_0 = test_images[i:i+1]
    layer_1 = relu(np.dot(layer_0,weights_0_1))
    layer_2 = np.dot(layer_1, weights_1_2)
    test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
    test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
  sys.stdout.write("\n" + \
      "I:" + str(j) + \
      " Test-Err:" + str(test_error/ float(len(test_images)))[0:5] +\
      " Test-Acc:" + str(test_correct_cnt/ float(len(test_images)))+\
      " Train-Err:" + str(error/ float(len(images)))[0:5] +\
      " Train-Acc:" + str(correct_cnt/ float(len(images))))


I:0 Test-Err:0.641 Test-Acc:0.6333 Train-Err:0.891 Train-Acc:0.413
I:1 Test-Err:1.219 Test-Acc:1.3168 Train-Err:0.650 Train-Acc:0.62
I:2 Test-Err:1.768 Test-Acc:2.0293 Train-Err:0.597 Train-Acc:0.669
I:3 Test-Err:2.298 Test-Acc:2.766 Train-Err:0.565 Train-Acc:0.686
I:4 Test-Err:2.790 Test-Acc:3.5268 Train-Err:0.547 Train-Acc:0.712
I:5 Test-Err:3.271 Test-Acc:4.2995 Train-Err:0.515 Train-Acc:0.739
I:6 Test-Err:3.748 Test-Acc:5.085 Train-Err:0.514 Train-Acc:0.727
I:7 Test-Err:4.210 Test-Acc:5.8661 Train-Err:0.499 Train-Acc:0.73
I:8 Test-Err:4.666 Test-Acc:6.6564 Train-Err:0.483 Train-Acc:0.759
I:9 Test-Err:5.130 Test-Acc:7.4433 Train-Err:0.469 Train-Acc:0.754
I:10 Test-Err:0.458 Test-Acc:0.787 Train-Err:0.472 Train-Acc:0.764
I:11 Test-Err:0.898 Test-Acc:1.5854 Train-Err:0.460 Train-Acc:0.779
I:12 Test-Err:1.349 Test-Acc:2.3904 Train-Err:0.465 Train-Acc:0.793
I:13 Test-Err:1.790 Test-Acc:3.1969 Train-Err:0.444 Train-Acc:0.803
I:14 Test-Err:2.232 Test-Acc:4.0001 Train-Err:0.458 Train-Acc:

In [18]:
#Batch gradient

In [22]:
np.random.seed(1)
def relu(x):
  return (x >= 0) * x
def relu2deriv(output):
  return output >= 0

batch_size = 100
alpha, iterations = (0.001, 300)
pixels_per_image, num_labels, hidden_size = (784, 10, 100)
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
  error, correct_cnt = (0.0, 0)
  for i in range(int(len(images) / batch_size)):
    batch_start, batch_end = ((i * batch_size),((i+1)*batch_size))

    layer_0 = images[batch_start:batch_end]
    layer_1 = relu(np.dot(layer_0,weights_0_1))

    dropout_mask = np.random.randint(2,size=layer_1.shape)
    
    layer_1 *= dropout_mask * 2
    layer_2 = np.dot(layer_1,weights_1_2)
    error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2)

    for k in range(batch_size):
      correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start+k:batch_start+k+1]))
      layer_2_delta = (labels[batch_start:batch_end]-layer_2) /batch_size
      layer_1_delta = layer_2_delta.dot(weights_1_2.T)*relu2deriv(layer_1)
      layer_1_delta *= dropout_mask
      weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
      weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
  if(j%10 == 0):
    test_error = 0.0
    test_correct_cnt = 0
    for i in range(len(test_images)):
      layer_0 = test_images[i:i+1]
      layer_1 = relu(np.dot(layer_0,weights_0_1))
      layer_2 = np.dot(layer_1, weights_1_2)

KeyboardInterrupt: ignored